### TSS mapping -1
- Add the name of the gene to the depth file


In [1]:
# Import the required libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load the data
depth_KO1 = pd.read_csv("depth/depth_sorted_trimmed_MBDSK1-cap_S39_R1_001.txt", delimiter = "\t", names = ["Chr", "Position", "Depth"], usecols = ["Position", "Depth"])
depth_KO2 = pd.read_csv("depth/depth_sorted_trimmed_MBDSK2-cap_S40_R1_001.txt", delimiter = "\t", names = ["Chr", "Position", "Depth"], usecols = ["Position", "Depth"])
depth_KO3 = pd.read_csv("depth/depth_sorted_trimmed_MBDSK3-cap_S41_R1_001.txt", delimiter = "\t", names = ["Chr", "Position", "Depth"], usecols = ["Position", "Depth"])
depth_WT1 = pd.read_csv("depth/depth_sorted_trimmed_MBWT1-cap_S36_R1_001.txt", delimiter = "\t", names = ["Chr", "Position", "Depth"], usecols = ["Position", "Depth"])
depth_WT2 = pd.read_csv("depth/depth_sorted_trimmed_MBWT2-cap_S37_R1_001.txt", delimiter = "\t", names = ["Chr", "Position", "Depth"], usecols = ["Position", "Depth"])
depth_WT3 = pd.read_csv("depth/depth_sorted_trimmed_MBWT3-cap_S38_R1_001.txt", delimiter = "\t", names = ["Chr", "Position", "Depth"], usecols = ["Position", "Depth"])

depth_KO1.head(10)  # Data is properly loaded

,Position,Depth
0,1,4
1,2,10
2,3,13
3,4,14
4,5,16
5,6,18
6,7,19
7,8,22
8,9,22
9,10,24


- Normalization step as in the shortRNAs paper

In [3]:
# # Check the total depth in all samples. 
# total_depth_KO1 = depth_KO1["Depth"].sum()
# total_depth_KO2 = depth_KO2["Depth"].sum()
# total_depth_KO3 = depth_KO3["Depth"].sum()
# total_depth_WT1 = depth_WT1["Depth"].sum()
# total_depth_WT2 = depth_WT2["Depth"].sum()
# total_depth_WT3 = depth_WT3["Depth"].sum()

# # Create a dictionary to store the name of the data -> depth sum
# depth_sums = {'KO1': total_depth_KO1, 'KO2': total_depth_KO2, 'KO3': total_depth_KO3, 'WT1': total_depth_WT1, 'WT2': total_depth_WT2, 'WT3': total_depth_WT3}

# # Make a bar plot
# plt.bar(depth_sums.keys(), depth_sums.values())
# plt.title("Bar plot of the total depth across samples")
# plt.xlabel("Data sets")
# plt.ylabel("Depth sum")
# plt.show()

# depth_sums.values()

# # In this plot it can be clearly seen how the total depth across samples varies, that's why we need to normalize the reads. 
# # The aim is that instead of having raw depth values, we want relative RNA abundance at each position, which is: (depth_at_position_x/total_depth) * 100

# depth_KO1["Nor_Depth"] = (depth_KO1["Depth"] / total_depth_KO1) * 100
# depth_KO2["Nor_Depth"] = (depth_KO2["Depth"] / total_depth_KO2) * 100
# depth_KO3["Nor_Depth"] = (depth_KO3["Depth"] / total_depth_KO3) * 100
# depth_WT1["Nor_Depth"] = (depth_WT1["Depth"] / total_depth_WT1) * 100
# depth_WT2["Nor_Depth"] = (depth_WT2["Depth"] / total_depth_WT2) * 100
# depth_WT3["Nor_Depth"] = (depth_WT3["Depth"] / total_depth_WT3) * 100

# # Check the total depth in all samples. 
# total_nor_depth_KO1 = depth_KO1["Nor_Depth"].sum()
# total_nor_depth_KO2 = depth_KO2["Nor_Depth"].sum()
# total_nor_depth_KO3 = depth_KO3["Nor_Depth"].sum()
# total_nor_depth_WT1 = depth_WT1["Nor_Depth"].sum()
# total_nor_depth_WT2 = depth_WT2["Nor_Depth"].sum()
# total_nor_depth_WT3 = depth_WT3["Nor_Depth"].sum()

# # Create a dictionary to store the name of the data -> depth sum
# depth_nor_sums = {'KO1': total_nor_depth_KO1, 'KO2': total_nor_depth_KO2, 'KO3': total_nor_depth_KO3, 'WT1': total_nor_depth_WT1, 'WT2': total_nor_depth_WT2, 'WT3': total_nor_depth_WT3}

# # Make a bar plot
# plt.bar(depth_nor_sums.keys(), depth_nor_sums.values())
# plt.title("Bar plot of the total normalized depth across samples")
# plt.xlabel("Data sets")
# plt.ylabel("Depth sum")
# plt.show()

# depth_nor_sums.values()

It can be seen how now the data has been normalized and the each group have the same total of reads

- Load the annotation file and add the genes in the depth files

In [10]:
# Load the gff file:
anot_file = pd.read_csv("Ref_genome/sequence.gff", sep = "\t", comment="#", names = ["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"], skiprows=4)
anot_file.to_excel("Data_outputs/raw_anot_file.xlsx")

# Check if in the column type threre are rows that are duplicated:
print(f"The total length of the anotation file is {len(anot_file)}. \n{anot_file['start'].duplicated().sum()} start values are duplicated. \n{anot_file['end'].duplicated().sum()} end values are duplicated")

# We see that not all rows are duplicated
print(anot_file["type"].value_counts())  # see there are other factors, which can have other positions w.r.t to the gene 

# In this experiment we are going to keep the CDS and the locus tag and genes associated with those. 
anot_file = anot_file[anot_file["type"] != "gene"]
# anot_file.head(10)

# Extract gene name if present, otherwise extract locus tag
anot_file["gene_name"] = anot_file["attributes"].str.extract(r'gene=([^;]+);', expand=False)

# If gene_name is NaN, use the locus tag instead
anot_file["gene_name"] = anot_file["gene_name"].fillna(anot_file["attributes"].str.extract(r'locus_tag=([^;]+)', expand=False))

# Remove the part of the locus_tag starting with 'BQ2027_'
anot_file['gene_name'] = anot_file['gene_name'].str.replace(r'BQ2027_', '', regex=True)

print(f"Are all values filled with gene names? {anot_file['gene_name'].notna().all()}")

# Drop non-important columns:
anot_file = anot_file.drop(columns=["seqid", "source", "score", "phase", "attributes"])

# Get an excel file
# anot_file = anot_file.drop(anot_file.columns[0], axis = 1)
anot_file.to_excel("Data_outputs/anot_file.xlsx")

anot_file.head(10)

The total length of the anotation file is 8296. 
4106 start values are duplicated. 
4106 end values are duplicated
gene              4097
CDS               4000
repeat_region      107
tRNA                45
mobile_element      42
rRNA                 3
misc_RNA             2
Name: type, dtype: int64
Are all values filled with gene names? False


,type,start,end,strand,gene_name
1,CDS,1,1524,+,dnaA
3,CDS,2052,3260,+,dnaN
5,CDS,3280,4437,+,recF
7,CDS,4434,4997,+,MB0004
9,CDS,5123,7267,+,gyrB
11,CDS,7302,9818,+,gyrA
13,CDS,9914,10828,+,MB0007
15,tRNA,10887,10960,+,ILET
17,tRNA,11112,11184,+,ALAT
19,CDS,11874,12311,-,MB0008c


In [8]:
def add_gene_names(data, anot_file):
    """Return the depth data set with the genes"""
    
    # Check if all start & end positions are in the depth file 
    # print(anot_file["start"].isin(data["Position"]).all())
    # print(anot_file["end"].isin(data["Position"]).all())

    # Add the gene name for the starting position:
    # Set the start points of the annotation file as indexes. 
    anot_file_sta_indexes = anot_file.set_index("start")
    anot_file_end_indexes = anot_file.set_index("end")
    # Extract the start positions as a list:
    start_pos = anot_file["start"].to_list()
    end_pos = anot_file["end"].to_list()
    
    for start_val in start_pos:
        data.loc[start_val -1, "Gene_name"] = anot_file_sta_indexes.loc[start_val, "gene_name"] + "_start"
    
    for end_val in end_pos:
        data.loc[end_val -1, "Gene_name"] = anot_file_end_indexes.loc[end_val, "gene_name"] + "_end"

    for _, row in anot_file.iterrows():  
        # print(row["start"])
        # Assign gene name for positions between start and end (inclusive)
        data.loc[(data["Position"] > row["start"]) & (data["Position"] < row["end"]), "Gene_name"] = row["gene_name"]
        
    data["Gene_name"] = data["Gene_name"].fillna("no_gene")    

    return data

gene_depth_KO1 = add_gene_names(depth_KO1, anot_file)
gene_depth_KO1.to_csv("Data_outputs/Depth_and_genes/genes_depth_KO1.txt", index = False)
gene_depth_KO2 = add_gene_names(depth_KO2, anot_file)                                              ## Run this when the TSS functions work properly
gene_depth_KO2.to_csv("Data_outputs/Depth_and_genes/genes_depth_KO2.txt", index = False)
gene_depth_KO3 = add_gene_names(depth_KO3, anot_file)
gene_depth_KO3.to_csv("Data_outputs/Depth_and_genes/genes_depth_KO3.txt", index = False)
gene_depth_WT1 = add_gene_names(depth_WT1, anot_file)
gene_depth_WT1.to_csv("Data_outputs/Depth_and_genes/genes_depth_WT1.txt", index = False)
gene_depth_WT2 = add_gene_names(depth_WT2, anot_file)
gene_depth_WT2.to_csv("Data_outputs/Depth_and_genes/genes_depth_WT2.txt", index = False)
gene_depth_WT3 = add_gene_names(depth_WT3, anot_file)
gene_depth_WT3.to_csv("Data_outputs/Depth_and_genes/genes_depth_WT3.txt", index = False)